<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex10/karen_rosero/Aula_10_Exerc%C3%ADcio_Karen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Karen Rosero"
print(f'Meu nome é {nome}')

Meu nome é Karen Rosero


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook
from collections import OrderedDict

In [4]:
# Check which GPU we are using
!nvidia-smi

Wed Jun  8 20:20:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:02:00.0  On |                  N/A |
| 32%   48C    P2    40W / 250W |   6673MiB / 12063MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva seu código aqui
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.features = self._token_window(texts)

    def _token_window(self, texts):
        feat = []
        y = []
        # bucle para ler cada uma das frases de entrada

        # Dataloader inspirado no notebook do Pedro Gengo
        for text in texts:
          # tokeniza uma frase
          tokens_from_text = tokenize(f'[CLS]{text}', self.tokenizer)
          tokens_from_text +=  [tokenizer.vocab['[PAD]']] * max(0, 1 + self.max_seq_length - len(tokens_from_text))
          for i in range(0, len(tokens_from_text)-1, self.max_seq_length): 
            if i+self.max_seq_length < len(tokens_from_text):
              feat.append(tokens_from_text[i:i+self.max_seq_length+1])
            else:
              feat.append(tokens_from_text[-self.max_seq_length-1:])
        return torch.tensor(feat).long()

    def __len__(self):
        # Escreva seu código aqui
        return len(self.features)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        feat = self.features[idx]
        #print(feat)
        return feat[:-1], feat[1:]

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))
#print('input',first_batch_input)
#print('target',first_batch_target)
correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

File ‘sample-1gb.txt’ already there; not retrieving.



In [9]:
# Load datasets
max_seq_length = 9

train_examples = 500
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
#print(f'Truncating to {max_lines} lines.')
#texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.


In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 31158155
valid examples: 14755
test examples: 8302


In [11]:
class MultiHead(torch.nn.Module):

  def __init__(self, max_seq_length: int, dim: int, heads:int, pad_token_id:int):
    super().__init__()
    self.max_seq_length = max_seq_length
    self.heads = heads
    self.pad_token_id = pad_token_id

    # Linear projections, input D output D
    self.W_Q = nn.Linear(dim, dim, bias=False)
    self.W_K = nn.Linear(dim, dim, bias=False)
    self.W_V = nn.Linear(dim, dim, bias=False)
    # Baseado na criação da função generate_square_subsequent_mask() 
    # https://jamesmccaffrey.wordpress.com/2020/12/16/generating-pytorch-transformer-masks/
    # crio a matriz com 1s da diagonal para baixo
    self.mask = torch.triu(torch.ones(self.max_seq_length, self.max_seq_length)).transpose(0, 1).unsqueeze(0).to(device)

  def attention(self, Q, K, V, mask):
    s_i = torch.matmul(Q, K.transpose(-2, -1))  
    # https://jamesmccaffrey.wordpress.com/2020/12/16/generating-pytorch-transformer-masks/
    s_i = s_i.masked_fill(mask.unsqueeze(1) == 0, -float("inf"))
    probs = nn.functional.softmax(s_i, dim=-2)  
    output = torch.matmul(probs, V).transpose(1, 2) 
    return output 
  

  def forward(self, inputs, mask):
    batch_size = inputs.shape[0]
    q = self.W_Q(inputs)  
    q = q.view(batch_size, self.max_seq_length, self.heads, -1)  
    q = q.transpose(1, 2)  
    k = self.W_K(inputs) 
    k = k.view(batch_size, self.max_seq_length, self.heads, -1)
    k = k.transpose(1, 2)  
    v = self.W_V(inputs) 
    v = v.view(batch_size, self.max_seq_length, self.heads, -1)
    v = v.transpose(1, 2)  
    
    output = self.attention(q, k, v, mask)
    output = output.reshape(batch_size, self.max_seq_length, -1)  
    
    return output  

In [12]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()
        self.max_seq_length = max_seq_length
        self.n_layers = n_layers
        self.pad_token_id = pad_token_id 
        self.embedding_ = nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
        self.embedding_P = nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)
        self.heads = MultiHead(max_seq_length, dim, n_layers, pad_token_id)
        self.W_O = torch.nn.Linear(dim, dim, bias = False)
        self.dense_layers = torch.nn.Sequential(
            OrderedDict([
                         ('L1', torch.nn.Linear(dim, dim)),
                         ('relu', torch.nn.ReLU()),
                         ('L2', torch.nn.Linear(dim, vocab_size, bias=False))
            ])
        )

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0]

        embds = self.embedding_(inputs) + self.embedding_P(inputs)  
        
        mask = torch.triu(torch.ones(batch_size, self.max_seq_length, self.max_seq_length)).transpose(1, 2).to(device)
        mask = mask.masked_fill(inputs.unsqueeze(1) == self.pad_token_id, 0)
        mask = mask.masked_fill(inputs.unsqueeze(2) == self.pad_token_id, 0)

        out = self.heads.forward(embds, mask)  
        out = self.W_O(out)

        outputL = self.dense_layers(out) 
        return outputL


## Teste o modelo com um exemplo

In [13]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5740992


## Assert da Perplexidade


In [15]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              29893
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [16]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=128, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=128)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29781.80, valid ppl: 29779.30
10000 steps; 1280000 examples so far; train ppl: 874.72, valid ppl: 567.15
20000 steps; 2560000 examples so far; train ppl: 426.70, valid ppl: 410.54
30000 steps; 3840000 examples so far; train ppl: 343.27, valid ppl: 355.32
40000 steps; 5120000 examples so far; train ppl: 306.42, valid ppl: 322.42
50000 steps; 6400000 examples so far; train ppl: 283.74, valid ppl: 303.20
60000 steps; 7680000 examples so far; train ppl: 266.94, valid ppl: 286.62
70000 steps; 8960000 examples so far; train ppl: 254.58, valid ppl: 274.56
80000 steps; 10240000 examples so far; train ppl: 244.75, valid ppl: 264.71
90000 steps; 11520000 examples so far; train ppl: 236.30, valid ppl: 256.49
100000 steps; 12800000 examples so far; train ppl: 229.19, valid ppl: 249.07
110000 steps; 14080000 examples so far; train ppl: 223.30, valid ppl: 243.34
120000 steps; 15360000 examples so far; train ppl: 218.09, valid ppl: 238.51
130000 steps; 16640000 

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [17]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 122.79104507575575


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [18]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com os
Eu gosto de comer pizza pois me faz com os que
Eu gosto de comer pizza pois me faz com os que a
Eu gosto de comer pizza pois me faz com os que a sua
Eu gosto de comer pizza pois me faz com os que a sua vida
Eu gosto de comer pizza pois me faz com os que a sua vida,
Eu gosto de comer pizza pois me faz com os que a sua vida, e
Eu gosto de comer pizza pois me faz com os que a sua vida, e a
Eu gosto de comer pizza pois me faz com os que a sua vida, e a ser
Eu gosto de comer pizza pois me faz com os que a sua vida, e a ser humano
Eu gosto de comer pizza pois me faz com os que a sua vida, e a ser humano que
Eu gosto de comer pizza pois me faz com os que a sua vida, e a ser humano que,
Eu gosto de comer pizza pois me faz com os que a sua vida, e a ser humano que, a
Eu gosto de comer pizza pois me faz com os que a sua vida, e a ser humano que, a ser
Eu gosto de comer pizza pois me faz com os que a sua vida, e a 

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.